# Partitioned Heritability

**Created**: 3 July 2022

## Environment

In [43]:
library(tidyverse)
library(ComplexHeatmap)
library(circlize)

setwd("~/eQTL_pQTL_Characterization/")

source("03_Functional_Interpretation/scripts/utils/ggplot_theme.R")

## Load Data

In [44]:
var.comps <- read.csv("~/gains_team282/epigenetics/enrichment/partitioned_heritability/variance_components.csv")

In [45]:
head(var.comps)

,Eigengene,Annotation,Component,Variance,Proportion
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,ME_1_1,Bulk_B-Control,Individual,5.443220e-08,1.143616e-02
2,ME_1_1,Bulk_B-Control,Annotation,4.120183e-16,8.656468e-11
3,ME_1_1,Bulk_B-Control,Other,1.198100e-06,2.517198e-01
4,ME_1_1,Bulk_B-Control,Residual,3.507126e-06,7.368440e-01
5,ME_2_1,Bulk_B-Control,Individual,0.000000e+00,0.000000e+00
6,ME_2_1,Bulk_B-Control,Annotation,0.000000e+00,0.000000e+00


In [46]:
lineages <- read.csv("03_Functional_Interpretation/data/Calderon_et_al_lineages.csv") %>%
    dplyr::bind_rows(c("Lineage"="MYELOID", "Cell_Type"="Neutrophils"))

In [47]:
head(lineages)

,Lineage,Cell_Type
,<chr>,<chr>
1,B,Bulk_B
2,B,Mem_B
3,B,Naive_B
4,B,Plasmablasts
5,CD8,CD8pos_T
6,CD8,Central_memory_CD8pos_T


## Heatmap of All Eigengenes

In [149]:
control.prop.mtx <- var.comps %>%
    dplyr::select(Annot=Annotation, everything()) %>%
    dplyr::filter(Component == "Annotation") %>%
    dplyr::filter(!grepl("segments", Annot)) %>%
    dplyr::filter(!grepl("cCREs", Annot)) %>%
    dplyr::filter(!grepl("Neutrophils", Annot)) %>%
    dplyr::filter(grepl("-Control", Annot)) %>%
    dplyr::mutate(Cell_Type = gsub("-.*$", "", Annot)) %>%
    dplyr::select(Eigengene, Cell_Type, Proportion) %>%
    tidyr::spread(Cell_Type, Proportion)

rownames(control.prop.mtx) <- gsub("_", " ", gsub("_1$", "", control.prop.mtx$Eigengene))
control.prop.mtx$Eigengene <- NULL
control.prop.mtx <- as.matrix(control.prop.mtx)

In [150]:
treated.prop.mtx <- var.comps %>%
    dplyr::select(Annot=Annotation, everything()) %>%
    dplyr::filter(Component == "Annotation") %>%
    dplyr::filter(!grepl("segments", Annot)) %>%
    dplyr::filter(!grepl("cCREs", Annot)) %>%
    dplyr::filter(!grepl("Neutrophils", Annot)) %>%
    dplyr::filter(grepl("-Stimulated", Annot)) %>%
    dplyr::mutate(Cell_Type = gsub("-.*$", "", Annot)) %>%
    dplyr::select(Eigengene, Cell_Type, Proportion) %>%
    tidyr::spread(Cell_Type, Proportion)

rownames(treated.prop.mtx) <- gsub("_", " ", gsub("_1$", "", treated.prop.mtx$Eigengene))
treated.prop.mtx$Eigengene <- NULL
treated.prop.mtx <- as.matrix(treated.prop.mtx)

In [151]:
prop.mtx <- var.comps %>%
    dplyr::select(Annot=Annotation, everything()) %>%
    dplyr::filter(Component == "Annotation") %>%
    dplyr::filter(grepl("Neutrophils", Annot)) %>%
    dplyr::mutate(Treatment = gsub("Neutrophils-", "", Annot)) %>%
    dplyr::select(Eigengene, Treatment, Proportion) %>%
    tidyr::spread(Treatment, Proportion)

rownames(prop.mtx) <- gsub("_", " ", gsub("_1$", "", prop.mtx$Eigengene))
prop.mtx$Eigengene <- NULL
prop.mtx <- as.matrix(prop.mtx)

In [152]:
neutrophil.map <- c(
    "BGP" = "Stimulation", "Control" = "Stimulation", "FLAG" = "Stimulation", "HMGB1" = "Stimulation", "LPS" = "Stimulation", "LTA" = "Stimulation", "R848" = "Stimulation",
    "SA-1" = "S. aureus", "SA-5" = "S. aureus", "WB" = "S. aureus",
    "EC1h" = "E. coli", "EC4h" = "E. coli", "noEC1h" = "E. coli", "noEC4h" = "E. coli"
)

In [157]:
options(repr.plot.width=15, repr.plot.height=24)

col_fun = colorRamp2(c(0, 1), c("white", "royalblue4"))

h1 = Heatmap(
    control.prop.mtx, name="Heritability", 
    col=col_fun, show_column_dend=F, show_row_dend=F, na_col="white",
    column_split=lineages$Lineage[match(colnames(control.prop.mtx), lineages$Cell_Type)]
)

h2 = Heatmap(
    treated.prop.mtx, name="Heritability",
    col=col_fun, show_column_dend=F, show_row_dend=F, na_col="white",
    column_split=lineages$Lineage[match(colnames(treated.prop.mtx), lineages$Cell_Type)]
)

h3 = Heatmap(
    prop.mtx, name="Heritability", 
    col=col_fun, show_column_dend=F, show_row_dend=F, na_col="white",
    column_split=neutrophil.map[colnames(prop.mtx)]
)

pdf("03_Functional_Interpretation/results/partitioned_heritability.pdf", width=16, height=24)
h1 + h2 + h3
dev.off()

Warning message:
“Heatmap/annotation names are duplicated: Heritability”
Warning message:
“Heatmap/annotation names are duplicated: Heritability, Heritability”


png 
  2